In [61]:
import sys
import base64
import random
import chardet
import codecs
import json

import numpy as np

from re import sub
from collections import defaultdict

from collections import defaultdict, Counter

import pickle

from sklearn.feature_extraction import DictVectorizer

In [62]:
def read_x(path, dv, max_count = None):
    f = open(path, "r")
    dict_list  = split.get_features(f.read(), max_count)
    
    
    
    return dv.fit_transform(dict_list), dv
    

In [582]:
def read_y(path, max_count = None):
    f = open(path, "r")
    result = []
   
    
    p = f.readline()
    
    total_read = 0
    while p != '':
        p = json.loads(p)
     
        for sentence in p["Sentences"]:
            n = sentence.count('.')
            if sentence[-1] == '.':
                n -= 1
            result += [0] * n
            if sentence[-1] not in ['!', '?']:
                result.append(1)
                
            total_read += n
            
            if max_count is not None and total_read >= max_count:
                return np.array(result)
        p = f.readline()
            
    
    return np.array(result)

In [583]:
import split
reload(split)

<module 'split' from 'split.pyc'>

In [584]:
# x, dv = read_x("gold.txt", DictVectorizer(sparse=False), max_count=None)

In [585]:
from sklearn.tree import DecisionTreeClassifier

In [586]:
y = read_y("gold.json", max_count = None)

In [587]:
len(y)

386

In [588]:
len(x)

380

In [589]:
y

array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1,